# Importing necessary libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import classification_report
import seaborn as sns 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve,auc
from sklearn.calibration import calibration_curve
import import_ipynb
from DataPreProcessing import new_df,X,y,X_train, X_test, y_train,y_test,sc

 # Building and Training a Neural Network Model

In [3]:
model = keras.Sequential([
    keras.layers.Dense(20, input_shape=(4,), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics=['accuracy'])
model.fit(X_train,y_train, epochs=5)

Epoch 1/5


## To Check if the model is overfitting or underfitting

In [ ]:
 
y_train_pred_ann = model.predict(X_train)
y_val_pred_ann = model.predict(X_test)

# Convert predictions to binary format (0 or 1)
y_train_pred_ann_binary = np.where(y_train_pred_ann > 0.5, 1, 0)
y_test_pred_ann_binary = np.where(y_val_pred_ann > 0.5, 1, 0)

# Calculate accuracy scores
train_accuracy_ann = accuracy_score(y_train, y_train_pred_ann_binary)
val_accuracy_ann = accuracy_score(y_test, y_test_pred_ann_binary)

In [ ]:
train_accuracy = round(train_accuracy_ann)
val_accuracy = round(val_accuracy_ann)
print("Training Accuracy:", train_accuracy )
print("Validation Accuracy:",  val_accuracy)


In [ ]:
if train_accuracy > val_accuracy:
    print("The model may be overfitting.")
elif train_accuracy < val_accuracy:
    print("The model may be underfitting.")
else:
    print("The model's performance on training and validation sets are similar")


# Predicting a new result

In [ ]:
model.predict([[1,9839.64, 170136.0,160296.36]])

In [ ]:
model.predict([[4,181.00, 181.00,0.00]])

# Prediciting the test set results

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
threshold = 0.5   #A threshold of 0.5 is often chosen when there is no specific preference for precision or recall. 
                  #It provides a balanced trade-off between the two metrics.
y_pred_binary = np.where(y_pred >= threshold, 1, 0)

In [ ]:
print(np.concatenate((y_pred_binary.reshape(len(y_pred_binary), 1),y_test.reshape(len(y_test), 1)),1))

# Accuracy

In [ ]:
accuracy = accuracy_score(y_test,y_pred_binary)
round(accuracy*100,2)

# Classification Report

In [ ]:
print(classification_report(y_test, y_pred_binary))

'''
0- No Fraud
1- Fraud
'''

# ROC Curve

In [ ]:
 
# Make predictions on the test set
y_pred_proba = model.predict(X_test)

# Calculate ROC AUC score
ann_roc_auc = roc_auc_score(y_test, y_pred_proba)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ANN Classifier (area = %0.2f)' % ann_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('ANN_ROC')
plt.show()


# Precision-Recall Curve

In [ ]:
y_prob_train_ann = model.predict(X_train)

# Calculate precision-recall curve
precision_ann, recall_ann, _ = precision_recall_curve(y_train, y_prob_train_ann)

# Plot precision-recall curve
plt.fill_between(recall_ann, precision_ann)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Train Precision-Recall curve")
plt.show()
from sklearn.metrics import precision_recall_curve, auc
auc_prc = auc(recall_ann, precision_ann)
print("AUC-PRC:", auc_prc)

# Confusion Matrix

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_binary)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


# Calibration Curve

In [ ]:
# Get predicted probabilities for the training set
y_prob_train_ann = model.predict(X_train)

# Calculate calibration curve
prob_true, prob_pred = calibration_curve(y_train, y_prob_train_ann, n_bins=10)

# Plot calibration curve
plt.plot(prob_pred, prob_true, marker='o', linestyle='-', label='Calibration Curve')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly calibrated')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Calibration Curve')
plt.legend()
plt.show()
